In [22]:
#Importing all the Necessary Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter("ignore", UserWarning)

### Load the dataset  
We first load the combined dataset from Excel into a pandas DataFrame.

In [23]:
file_path = r"C:\Users\kavit\Documents\Git\Team6_InsightHunters_PyhtonHackathon_August2025\Consolidated_Excel.xlsx"
diabts_df = pd.read_excel(file_path, engine="openpyxl")
diabts_df.head()

,Patient ID,time,glucose,calories,heart_rate,steps,basal_rate,bolus_volume_delivered,carb_input,Age,Gender,Race,Average Sleep Duration (hrs),Sleep Quality (1-10),% with Sleep Disturbances
0,HUPA0001P,2018-06-13T18:40:00,332.0,6.3595,82.322835,34,0.091667,0.0,0.0,34,Male,Other,6.3,4.5,80
1,HUPA0001P,2018-06-13T18:45:00,326.0,7.7280,83.740157,0,0.091667,0.0,0.0,34,Male,Other,6.3,4.5,80
2,HUPA0001P,2018-06-13T18:50:00,330.0,4.7495,80.525180,0,0.091667,0.0,0.0,34,Male,Other,6.3,4.5,80
3,HUPA0001P,2018-06-13T18:55:00,324.0,6.3595,89.129032,20,0.091667,0.0,0.0,34,Male,Other,6.3,4.5,80
4,HUPA0001P,2018-06-13T19:00:00,306.0,5.1520,92.495652,0,0.075000,0.0,0.0,34,Male,Other,6.3,4.5,80


In [24]:
diabts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309392 entries, 0 to 309391
Data columns (total 15 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Patient ID                    309392 non-null  object 
 1   time                          309392 non-null  object 
 2   glucose                       309392 non-null  float64
 3   calories                      309392 non-null  float64
 4   heart_rate                    309392 non-null  float64
 5   steps                         309392 non-null  int64  
 6   basal_rate                    309392 non-null  float64
 7   bolus_volume_delivered        309392 non-null  float64
 8   carb_input                    309392 non-null  float64
 9   Age                           309392 non-null  int64  
 10  Gender                        309392 non-null  object 
 11  Race                          309392 non-null  object 
 12  Average Sleep Duration (hrs)  309392 non-nul

### Step 1 – Convert time column to datetime
The `time` column is currently a string. We convert it to a proper datetime format so we can do time-based analysis later.

In [25]:
diabts_df['time'] = pd.to_datetime(diabts_df['time'], errors='coerce')
diabts_df[['Patient ID', 'time']].head()

,Patient ID,time
0,HUPA0001P,2018-06-13 18:40:00
1,HUPA0001P,2018-06-13 18:45:00
2,HUPA0001P,2018-06-13 18:50:00
3,HUPA0001P,2018-06-13 18:55:00
4,HUPA0001P,2018-06-13 19:00:00


### Step 2: Convert numeric columns & format
- Convert columns to numeric, coercing invalid entries to NaN.  
- Force `glucose` and `heart_rate` to integers.  
- Round all other decimal columns to 2 decimal places.

In [26]:
num_cols = ["glucose", "calories", "heart_rate", "steps",
            "basal_rate", "bolus_volume_delivered", "carb_input",
            "Age", "Average Sleep Duration (hrs)",
            "Sleep Quality (1-10)", "% with Sleep Disturbances"]

# Convert all to numeric (invalid entries become NaN)
for col in num_cols:
    diabts_df[col] = pd.to_numeric(diabts_df[col], errors='coerce')

# Force integer columns using nullable Int64
diabts_df["glucose"] = diabts_df["glucose"].round(0).astype("Int64")      
diabts_df["heart_rate"] = diabts_df["heart_rate"].round(0).astype("Int64")  

# Round all other decimal columns to 2 decimal places
decimal_cols = ["calories", "basal_rate", "bolus_volume_delivered", "carb_input",
                "Average Sleep Duration (hrs)", "Sleep Quality (1-10)", "% with Sleep Disturbances"]

for col in decimal_cols:
    diabts_df[col] = diabts_df[col].round(2)

### Step 3 – Standardize categorical demographic columns  
We clean up the `Gender` and `Race` columns by stripping spaces and normalizing case.

In [27]:
diabts_df['Gender'] = diabts_df['Gender'].astype(str).str.strip().str.capitalize()
diabts_df['Race'] = diabts_df['Race'].astype(str).str.strip().str.title()

diabts_df[['Patient ID', 'Gender', 'Race']].drop_duplicates().head()

,Patient ID,Gender,Race
0,HUPA0001P,Male,Other
4096,HUPA0002P,Male,Hispanic
7277,HUPA0003P,Male,Black
11047,HUPA0004P,Female,Native American
14231,HUPA0005P,Male,Native American


### Step 4 – Remove duplicate rows  
Some patients may have duplicate records for the same timestamp.  
We drop duplicates based on `Patient ID` and `time`.


In [28]:
before = len(diabts_df)
diabts_df = diabts_df.drop_duplicates(subset=["Patient ID", "time"])
after = len(diabts_df)

print(f"✅ Removed {before - after} duplicate rows")

✅ Removed 0 duplicate rows


### Step 5: Check for negative values in calories, steps, basal_rate, heart rate

In [29]:
# Columns to check
cols_to_check = ['calories', 'steps', 'basal_rate', 'heart_rate']

# Check for negative values
negatives = {}
for col in cols_to_check:
    negatives[col] = (diabts_df[col] < 0).sum()

# Display count of negative values per column
negatives_df = pd.DataFrame.from_dict(negatives, orient='index', columns=['Negative Count'])
print("Negative value counts:")
print(negatives_df)

# Convert negative values to positive
diabts_df[cols_to_check] = diabts_df[cols_to_check].abs()


Negative value counts:
            Negative Count
calories                 0
steps                    0
basal_rate               0
heart_rate               0


### Step 6 – Check for extreme outliers in glucose  
Glucose values below 40 or above 600 are physiologically unlikely and may be errors.  
We filter them out.

In [30]:
# Define realistic glucose limits
glucose_low = 40
glucose_high = 600

# Identify glucose outliers
glucose_outliers = diabts_df[(diabts_df['glucose'] < glucose_low) | (diabts_df['glucose'] > glucose_high)]

# Display count
print(f"Number of glucose outlier rows: {len(glucose_outliers)}")

# Optionally, display the rows
glucose_outliers


Number of glucose outlier rows: 0


,Patient ID,time,glucose,calories,heart_rate,steps,basal_rate,bolus_volume_delivered,carb_input,Age,Gender,Race,Average Sleep Duration (hrs),Sleep Quality (1-10),% with Sleep Disturbances


### Step 7: Check for Null percentage

In [31]:
null_percentage = diabts_df.isnull().mean() * 100
print(null_percentage)

Patient ID                      0.0
time                            0.0
glucose                         0.0
calories                        0.0
heart_rate                      0.0
steps                           0.0
basal_rate                      0.0
bolus_volume_delivered          0.0
carb_input                      0.0
Age                             0.0
Gender                          0.0
Race                            0.0
Average Sleep Duration (hrs)    0.0
Sleep Quality (1-10)            0.0
% with Sleep Disturbances       0.0
dtype: float64


### Step 8: Extract Hour and Day of Week
We create two new columns from the `time` column:
- `hour_of_day`: Hour when glucose was recorded (0–23)
- `day_of_week`: Day of the week (Monday=0, Sunday=6)
These features can help analyze patterns in glucose levels and other metrics over the day and week.

In [32]:
diabts_df['hour_of_day'] = diabts_df['time'].dt.hour
diabts_df['day_of_week'] = diabts_df['time'].dt.dayofweek

### Step 9: Flag High Glucose Events
We create a binary column `high_glucose` that indicates whether glucose is above 180 mg/dL, which can be considered a high glucose event.

In [33]:
# Create high glucose flag
diabts_df['high_glucose'] = (diabts_df['glucose'] > 180).astype(int)

# Check
diabts_df[['Patient ID', 'time', 'glucose', 'high_glucose']].head(10)


,Patient ID,time,glucose,high_glucose
0,HUPA0001P,2018-06-13 18:40:00,332,1
1,HUPA0001P,2018-06-13 18:45:00,326,1
2,HUPA0001P,2018-06-13 18:50:00,330,1
3,HUPA0001P,2018-06-13 18:55:00,324,1
4,HUPA0001P,2018-06-13 19:00:00,306,1
5,HUPA0001P,2018-06-13 19:05:00,313,1
6,HUPA0001P,2018-06-13 19:10:00,312,1
7,HUPA0001P,2018-06-13 19:15:00,293,1
8,HUPA0001P,2018-06-13 19:20:00,303,1
9,HUPA0001P,2018-06-13 19:25:00,293,1


### Step 10: Flag extreme outliers in heart rate beyond realistic human limits

In [34]:
# Define realistic heart rate thresholds
min_hr = 30
max_hr = 200

# Filter heart rate outliers
heart_rate_outliers = diabts_df[(diabts_df['heart_rate'] < min_hr) | (diabts_df['heart_rate'] > max_hr)]

# Print the number of outliers
print(f"Number of heart rate outliers: {len(heart_rate_outliers)}")

# Display the outlier rows
heart_rate_outliers[['Patient ID', 'time', 'heart_rate']]

Number of heart rate outliers: 0


,Patient ID,time,heart_rate


In [35]:
diabts_df.head()

,Patient ID,time,glucose,calories,heart_rate,steps,basal_rate,bolus_volume_delivered,carb_input,Age,Gender,Race,Average Sleep Duration (hrs),Sleep Quality (1-10),% with Sleep Disturbances,hour_of_day,day_of_week,high_glucose
0,HUPA0001P,2018-06-13 18:40:00,332,6.36,82,34,0.09,0.0,0.0,34,Male,Other,6.3,4.5,80,18,2,1
1,HUPA0001P,2018-06-13 18:45:00,326,7.73,84,0,0.09,0.0,0.0,34,Male,Other,6.3,4.5,80,18,2,1
2,HUPA0001P,2018-06-13 18:50:00,330,4.75,81,0,0.09,0.0,0.0,34,Male,Other,6.3,4.5,80,18,2,1
3,HUPA0001P,2018-06-13 18:55:00,324,6.36,89,20,0.09,0.0,0.0,34,Male,Other,6.3,4.5,80,18,2,1
4,HUPA0001P,2018-06-13 19:00:00,306,5.15,92,0,0.08,0.0,0.0,34,Male,Other,6.3,4.5,80,19,2,1


### Step 11: Save the Cleaned and Preprocessed data frame to an excel file

In [36]:
import os

# Set output path (change folder if needed)
output_file = os.path.join(r"C:\Users\kavit\OneDrive\Documents\PYTHON", "diabetes_cleaned.xlsx")

# Save cleaned dataframe to Excel
diabts_df.to_excel(output_file, index=False, engine='openpyxl')

print(f"✅ Cleaned dataset saved to: {output_file}")


✅ Cleaned dataset saved to: C:\Users\kavit\OneDrive\Documents\PYTHON\diabetes_cleaned.xlsx
